<img src="https://justaklikaway.files.wordpress.com/2014/05/shut-up-and-take-my-money.jpg" height="400" width="400"> 

# <center> ML для маркетинга. <br>  <br> Cюжетная арка 1: продажи </center>

В этом семестре мы с вами пройдём через несколько сюжетных арок. Первой такой аркой будут продажи. Им будет посвящена первая серия из семинаров и домашек. 


#  Эпизод III (месть менеджера) 

Ну вот и всё, модель готова. Она прогнозирует продажи. Когда нас попросили сделать её, предполагалось, что она будет решать какую-то реальную сложную проблему и сэкономит нам гору бабла. Давайте посмотрим насколько успешно она с этим справляется.

In [1]:
# Как обычно, для начала подгружаем нужные нам пакеты
import pandas as pd    # пакет для работы с таблицами 
import numpy as np     # пакет для работы с матрицами 

# пакеты для картиночек 
import matplotlib

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')  # правильный (наиболее красивый) стиль у графиков

%matplotlib inline

# 1. Модель и тестовы данные

Подгрузим данные, которые мы отложили для тестового периода. Их наша модель ещё не видела. Интересно, как именно она на них себя покажет.

In [2]:
df = pd.read_csv('sales_test_v1.tsv', sep='\t')    # подгружаем датасет
print('Размерность таблицы:', df.shape)      # shape показывает его размер
df.head()                    

Размерность таблицы: (73684, 148)


,Weekly_Sales,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,...,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Black_Friday,Pre_christmas
0,17147.44,151315,24.194444,3.749,21290.13,0.0,69.89,4977.35,3261.04,221.671800,...,0,0,0,1,0,0,0,0,0,0
1,18164.20,151315,23.205556,3.688,8351.40,0.0,10.52,2443.14,3127.88,221.725663,...,0,0,0,1,0,0,0,0,0,0
2,18517.79,151315,21.294444,3.630,6154.14,0.0,45.11,1675.49,5508.18,221.742674,...,0,0,0,1,0,0,0,0,0,0
3,16963.55,151315,25.122222,3.561,4039.39,0.0,745.19,1429.96,3631.13,221.744944,...,0,0,0,1,0,0,0,0,0,0
4,16065.49,151315,25.527778,3.501,6086.21,12.0,370.51,148.75,3690.85,221.747214,...,0,0,0,0,1,0,0,0,0,0


Подгружаем нашу модель. 

In [3]:
from sklearn.externals import joblib

model_lasso = joblib.load('model_regression.pkl')
model_lasso

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

Давайте построим небольшую табличку с прогнозами, с которой в дальнейшем и будем работать.

In [4]:
# это то, что мы пытаемся прогнозировать! 
y = df['ln_future_sales']

# то, по чему пытаемся прогнозировать, ln_Weekly_Sales оставили!
df = df.drop(['ln_future_sales', 'future_sales', 'Weekly_Sales'], axis=1)
X = df.get_values()

Создайте табличку `dfp` и положите в неё три колонки: `real` с настоящими продажами в тестовом периоде и `regression_predict` с прогнозными, `naive` с текущими продажами.

In [19]:
dfp = pd.DataFrame() 

# Экспонента, потому что переходим от логарифмов к деньгам
dfp['real'] = np.exp(y)
dfp['predict'] = np.exp(model_lasso.predict(X))
dfp['naive'] = np.exp(df['ln_Weekly_Sales'])

dfp.head()

,real,predict,naive
0,18165.20,18317.925422,17148.44
1,18518.79,19003.533617,18165.20
2,16964.55,19486.145112,18518.79
3,16066.49,18020.766871,16964.55
4,17667.00,17569.439838,16066.49


In [20]:
from sklearn.metrics import mean_absolute_error as mae
mae(dfp.real, dfp.predict)

1716.4399712236902

In [21]:
mae(dfp.real, dfp.naive)

1576.8757076163072

# 2. Наивный прогноз

Обычно модели учат, когда хотят что-нибудь улучшить. Давайте представим себе, что Walrmat нас с вами нанял, чтобы решить проблемы с оптимизацией закупок. 

* Будем считать, что мы работаем со скоропортящимся продуктом. Если люди в течение недели его не раскупают, он протухает. Приходится его выбрасывать.
* Будем считать, что если продуктов не хватает, магазин не может удовлетворить спрос клиента, но при этом записывает в книжечку сколько потенциально продуктов не было продано. 

Ну и последняя предпосылка. Магазин покупает на следущую неделю столько продуктов, сколько хотели купить потребители в течение текущей недели.

Сколько денег в течение тестового периода мы теряем из-за перепрогноза и того, что продукт портится? 

In [22]:
# вычитаем из прогнозных значений реальные
loss = dfp['naive'].values - dfp['real'].values
loss[:5]

array([-1016.76,  -353.59,  1554.24,   898.06, -1600.51])

In [23]:
loss > 0  # был ли перепрогноз

array([False, False,  True, ..., False,  True, False])

In [24]:
loss[loss > 0] # только потери при перепрогнозе

array([1554.24,  898.06,  107.18, ...,   69.36,  138.66,  301.01])

In [25]:
l_naive_over = np.sum(loss[loss > 0])  # суммарные денежные потери
l_naive_over 

57851127.08999999

Сколько денег в течение тестового периода мы теряем из-за недопрогноза?

In [26]:
l_naive_under = -1*np.sum(loss[loss < 0])
l_naive_under

58339382.54999999

Обратите внимание, что если потребитель пришёл в магазин и ему не хватает товара, он уходит. Мы не всегда можем точно узнать что именно он хотел купить, поэтому цифра недопрогноза на практике будет получатся примерной. 

# 3. Регрессия 

Теперь на арену выходит наша регрессия. Давайте представим себе, что мы внедрили её во всех магазинах. Какими будут наши потери от перепрогноза и недопрогноза? 

In [27]:
loss = dfp['predict'].values - dfp['real'].values
l_regr_over = np.sum(loss[loss > 0])
l_regr_under =-1*np.sum(loss[loss < 0])

l_regr_over, l_regr_under

(63291896.97219472, 63182265.86745168)

In [28]:
l_regr_over, l_naive_over

(63291896.97219472, 57851127.08999999)

In [29]:
l_regr_under, l_naive_under

(63182265.86745168, 58339382.54999999)

Вопрос в том, будет ли от этого экономия... 

In [30]:
l_naive_over - l_regr_over # экономия при перепрогнозах

-5440769.882194728

In [31]:
l_naive_under - l_regr_under # экономия при недопрогнозах

-4842883.317451693

Неожиданно оказывается, что прогноз __"завтра будет то же самое, что и сегодня"__ работает лучше. Пока что наша модель абсолютно бесполезна. Иногда так бывает. Для того, чтобы обучить хорошую модель, нужно лучше стараться. Как думаете, что мы сделали с вами при обучении плохо? 

Обратите внимание на следущую вещь. Обычно, когда модель работает, не ограничиваются подсчётом какого-то одного числа. На части магазинов запускают полноценный АБ-тест. В следущей сюжетной арке, когда мы доберёмся до применения, и моделька окажется хорошей, мы обязательно сымитируем такой тест.

## Почиташки 

* [Интересное чтиво](https://secretmag.ru/business/trade-secret/brilliant-v-smetane-kak-izbyonka-stala-prodavcom-svezhei-molochki-1-v-rossii.htm) про Избёнку 